In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='0'
import sys
import warnings
import gc
import random
import numpy as np
random.seed(31)
np.random.seed(23)
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.color import rgb2gray
from skimage.morphology import label
from scipy import ndimage as ndi
import zipfile

from keras.models import Model, load_model
from keras.layers import Input, Dropout, BatchNormalization, Add, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Activation, ZeroPadding2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras import backend as K

import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)
K.set_session(session)

# Set some parameters
BS = 32
EPOCHS_1 = 100
EPOCHS_2 = 100
ORI_D = 101
TRAIN_ZIP = '../input/train.zip'
TEST_ZIP = '../input/test.zip'
TRAIN_PATH = '../input/train/'
TEST_PATH = '../input/test/'
TRAIN_CSV = '../input/train.csv'
DEPTH_CSV = '../input/depths.csv'

if not os.path.exists(TRAIN_PATH):
    zip_ref = zipfile.ZipFile(TRAIN_ZIP, 'r')
    zip_ref.extractall(TRAIN_PATH)
    zip_ref.close()
if not os.path.exists(TEST_PATH):
    zip_ref = zipfile.ZipFile(TEST_ZIP, 'r')
    zip_ref.extractall(TEST_PATH)
    zip_ref.close()
if not os.path.exists('./models'):
    os.makedirs('./models')
print(os.listdir(TRAIN_PATH))

In [ ]:
def rle_encoding(x, order='F', to_str=True):
    dots = np.where(x.flatten(order=order) == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    if to_str:
        return ' '.join(str(y) for y in run_lengths)
    return run_lengths


In [ ]:
train_csv = pd.read_csv(TRAIN_CSV)
depth_csv = pd.read_csv(DEPTH_CSV)
test_csv = depth_csv[~depth_csv.index.isin(train_csv.index)]
gc.collect()

In [ ]:
X_train = []
Y_train = [] 

print('Getting train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_csv.id), total=len(train_csv.id)):
    path = TRAIN_PATH
    img = imread(path + 'images/' + id_ + '.png')[...,0].astype(np.uint8)
    assert img.ndim==2
    assert img.shape[0]==ORI_D and img.shape[1]==ORI_D
    X_train.append(img)
    mask = np.squeeze(imread(path + 'masks/' + id_ + '.png')>0).astype(np.uint8)
    assert mask.ndim==2
    Y_train.append(mask)
del img, mask
gc.collect()

In [ ]:
# Check if training data looks all right
ix = np.random.randint(0, len(train_csv.id))
plt.imshow(X_train[ix], cmap='gray')
plt.show()
plt.imshow(Y_train[ix], cmap='gray')
plt.show()

In [ ]:
def custom_loss(y_true, y_pred_):
    
    y_pred = K.sigmoid(y_pred_)
    
    def dice_coef(y_true, y_pred): 
        return (2.* K.sum(y_true*y_pred) + 1) / (K.sum(y_true) + K.sum(y_pred) + 1) # scaler
    
    s_dice_coef = dice_coef(y_true, y_pred) # scaler
    s_bce = K.mean(K.binary_crossentropy(y_true, y_pred)) # scaler
    loss = 0.5*s_bce - s_dice_coef + 1. # scaler [0, inf)
    
    return loss # scaler [0, inf)

In [ ]:
# Define IoU metric
def score_numpy(Ys, Ps, th=0.0):
    iou_ths = np.arange(0.0, 1.0, 0.05)
    Ps = Ps.reshape(Ps.shape[0], -1).copy()
    Ys = Ys.reshape(Ys.shape[0], -1).copy()
    for n, P in enumerate(Ps):
        Ps[n] = P>th
    Ps = Ps.astype(np.bool)
    Ys = Ys.astype(np.bool)
    TPs = np.zeros_like(iou_ths, dtype=np.float32)
    FPs = np.zeros_like(iou_ths, dtype=np.float32)
    FNs = np.zeros_like(iou_ths, dtype=np.float32)
    for Y, P in zip(Ys, Ps):
        iou = (Y&P).sum() / ((Y|P).sum()+1e-9)
        Y_sum = Y.sum()
        P_sum = P.sum()
        for n, iou_th in enumerate(iou_ths):
            if iou>=iou_th: # TP
                TPs[n]+=1
            elif Y_sum<=0 and P_sum>0: # FP
                FPs[n]+=1
            elif Y_sum>0 and P_sum<=0: # FN
                FNs[n]+=1
    return np.mean(TPs / (TPs+FPs+FNs+1e-9))

def score(label, pred):
    metric_value = tf.py_func(score_numpy, [label, pred], tf.float32)
    return metric_value

In [ ]:
# code download from: https://github.com/bermanmaxim/LovaszSoftmax
def lovasz_grad(gt_sorted):
    """
    Computes gradient of the Lovasz extension w.r.t sorted errors
    See Alg. 1 in paper
    """
    gts = tf.reduce_sum(gt_sorted)
    intersection = gts - tf.cumsum(gt_sorted)
    union = gts + tf.cumsum(1. - gt_sorted)
    jaccard = 1. - intersection / (union+1e-9)
    jaccard = tf.concat((jaccard[0:1], jaccard[1:] - jaccard[:-1]), 0)
    return jaccard


# --------------------------- BINARY LOSSES ---------------------------

def lovasz_hinge(logits, labels, per_image=True, ignore=None):
    """
    Binary Lovasz hinge loss
      logits: [B, H, W] Variable, logits at each pixel (between -\infty and +\infty)
      labels: [B, H, W] Tensor, binary ground truth masks (0 or 1)
      per_image: compute the loss per image instead of per batch
      ignore: void class id
    """
    if per_image:
        def treat_image(log_lab):
            log, lab = log_lab
            log, lab = tf.expand_dims(log, 0), tf.expand_dims(lab, 0)
            log, lab = flatten_binary_scores(log, lab, ignore)
            return lovasz_hinge_flat(log, lab)
        losses = tf.map_fn(treat_image, (logits, labels), dtype=tf.float32)
        loss = tf.reduce_mean(losses)
    else:
        loss = lovasz_hinge_flat(*flatten_binary_scores(logits, labels, ignore))
    return loss


def lovasz_hinge_flat(logits, labels):
    """
    Binary Lovasz hinge loss
      logits: [P] Variable, logits at each prediction (between -\infty and +\infty)
      labels: [P] Tensor, binary ground truth labels (0 or 1)
      ignore: label to ignore
    """

    def compute_loss():
        labelsf = tf.cast(labels, logits.dtype)
        signs = 2. * labelsf - 1.
        errors = 1. - logits * tf.stop_gradient(signs)
        errors_sorted, perm = tf.nn.top_k(errors, k=tf.shape(errors)[0], name="descending_sort")
        gt_sorted = tf.gather(labelsf, perm)
        grad = lovasz_grad(gt_sorted)
        loss = tf.tensordot(tf.nn.relu(errors_sorted), tf.stop_gradient(grad), 1, name="loss_non_void")
        return loss

    # deal with the void prediction case (only void pixels)
    loss = tf.cond(tf.equal(tf.shape(logits)[0], 0),
                   lambda: tf.reduce_sum(logits) * 0.,
                   compute_loss,
                   strict=True,
                   name="loss"
                   )
    return loss


def flatten_binary_scores(scores, labels, ignore=None):
    """
    Flattens predictions in the batch (binary case)
    Remove labels equal to 'ignore'
    """
    scores = tf.reshape(scores, (-1,))
    labels = tf.reshape(labels, (-1,))
    if ignore is None:
        return scores, labels
    valid = tf.not_equal(labels, ignore)
    vscores = tf.boolean_mask(scores, valid, name='valid_scores')
    vlabels = tf.boolean_mask(labels, valid, name='valid_labels')
    return vscores, vlabels

def lovasz_loss(y_true, y_pred):
    y_true, y_pred = K.cast(K.squeeze(y_true, -1), 'int32'), K.cast(K.squeeze(y_pred, -1), 'float32')
    #logits = K.log(y_pred / (1. - y_pred))
    logits = y_pred #Jiaxin
    loss = lovasz_hinge(logits, y_true, per_image = True, ignore = None)
    return loss

In [ ]:
def build_model():
    def conv(f, k=3, act='relu'):
        return Conv2D(f, (k, k), activation=act, kernel_initializer='he_normal', padding='same')
    def _res_conv(inputs, f, k=3): # very simple residual module
        channels = int(inputs.shape[-1])
        
        cs = inputs
        
        cs = BatchNormalization() (cs)
        cs = Activation('relu') (cs)
        cs = conv(f, 3, act=None) (cs)
        
        cs = Dropout(0.1) (cs)
        
        cs = BatchNormalization() (cs)
        cs = Activation('relu') (cs)
        cs = conv(f, 3, act=None) (cs)
        
        if f!=channels:
            t1 = conv(f, 1, None) (inputs) # identity mapping
        else:
            t1 = inputs
        
        out = Add()([t1, cs]) # t1 + c2
        return out
    def pool():
        return MaxPooling2D((2, 2))
    def up(inputs, pad='same'):
        upsampled = Conv2DTranspose(int(inputs.shape[-1]), (3, 3), strides=(2, 2), kernel_initializer='he_normal', padding=pad) (inputs)
        return upsampled
    
    inputs = Input(shape=(ORI_D, ORI_D, 1))
    preprocess1 = Lambda(lambda x: x/127.5-1.0) (inputs)
    
    r = 16
    rep = 4
    mid_rep = 3
    x = preprocess1
    
    skip_connections = []
    pad_mode = ['same', 'valid', 'same', 'valid']
    
    for t in range(rep):
        x = conv(r*int(2**t), 3, None) (x)
        x = _res_conv(x, r*int(2**t), 3)
        x = _res_conv(x, r*int(2**t), 3)
        x = BatchNormalization() (x)
        x = Activation('relu') (x)
        skip_connections.append(x)
        x = pool() (x)
    
    x = conv(r*int(2**rep), 3, None) (x)
    for t in range(mid_rep):
        x = _res_conv(x, r*int(2**rep))
    
    for t, s, p in zip(reversed(range(rep)), reversed(skip_connections), pad_mode):
        x = BatchNormalization() (x)
        x = Activation('relu') (x)
        x = up(x, p)
        x = concatenate([s, x])
        x = conv(r*int(2**t), 3, None) (x)
        x = _res_conv(x, r*int(2**t), 3)
        x = _res_conv(x, r*int(2**t), 3)
    x = BatchNormalization() (x)
    x = Activation('relu') (x)
    outputs = Conv2D(1, (1, 1), activation=None, kernel_initializer='he_normal', padding='valid') (x)
    return Model([inputs], [outputs])

In [ ]:
model = build_model()
model.summary()

In [ ]:
from keras.utils import Sequence
import cv2
from sklearn.utils import shuffle
from skimage.transform import AffineTransform, warp
import copy
import imgaug as ia
from imgaug import augmenters as iaa

class data_generator(Sequence):
    def __init__(self, data, label, batch_size=4, training=True, shuffle=True):
        self.data = data
        self.label= label
        self.batch_size = batch_size
        self.training = training
        self.shuffle = shuffle
        sometimes = lambda aug: iaa.Sometimes(0.5, aug)
        self.seq_color = iaa.SaltAndPepper((0.01, 0.05))
        self.distortion = iaa.Sequential([
                sometimes(iaa.Affine(
                    rotate=(-5, 5), # rotate by -45 to +45 degrees
                    order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                    cval=0, # if mode is constant, use a cval between 0 and 255
                    mode='constant' # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                )),
                iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05)),
                sometimes(iaa.ElasticTransformation(alpha=(0.01, 0.4), sigma=0.2)),
                sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.02)))
            ], random_order=True)
    def __len__(self):
        return int(np.ceil(float(len(self.data))/self.batch_size))
    def on_epoch_end(self):
        if self.shuffle: self.data, self.label = shuffle(self.data, self.label)
    def __getitem__(self, i):
        l_bound =  i    * self.batch_size
        r_bound = (i+1) * self.batch_size
        if r_bound>len(self.data): # ensure every iteration has the same batch size
            r_bound = len(self.data)
            l_bound = r_bound - self.batch_size
        dat_que = np.empty((self.batch_size, ORI_D, ORI_D, 1), dtype=np.uint8)
        lab_que = np.empty((self.batch_size, ORI_D, ORI_D, 1), dtype=np.uint8)
        for n, index in enumerate(range(l_bound, r_bound)):
            img = copy.deepcopy(self.data[index]).astype(np.float32) / 255.
            lab = copy.deepcopy(self.label[index]).astype(np.float32)
            if self.training:
                if np.random.rand() < .5: # flip horizontal
                    img = np.flip(img, 1)
                    lab = np.flip(lab, 1)

                if np.random.rand()<0.4:
                    a = .05 # amptitude
                    t  = np.random.uniform(1-a,1+a)
                    img = np.clip(img * t, 0, 1) 
                
                if np.random.rand()<0.3:
                    sigma = np.random.rand()*0.03
                    img = np.clip(img + np.random.randn(*img.shape)*sigma, 0, 1)
                
                if np.random.rand() < 0.3:
                    img = np.clip(self.seq_color.augment_image(img), 0, 1)
                if np.random.rand() < 0.3:
                    det = self.distortion.to_deterministic()
                    img = np.clip(det.augment_image(img), 0, 1)
                    lab = np.clip(det.augment_image(lab), 0, 1)
                if np.random.rand() < 0.1:
                    img = cv2.GaussianBlur(img, (3, 3), 0)
                
                if np.random.rand() < 0.3:
                    crop_ratio = np.random.uniform(0.01, 0.1, size=4)
                    u, r, d, l = np.round(crop_ratio * np.array([img.shape[0], img.shape[1]]*2)).astype(np.uint8)
                    img = img[u:-d,l:-r] # crop image
                    lab = lab[u:-d,l:-r] # crop image
            ### end of data augmentation ###

            img = np.clip(img.astype(np.float32)*255,0, 255).astype(np.uint8)
            lab = np.round(np.clip(lab.astype(np.float32),0, 1)).astype(np.uint8)
            
            if img.shape[0]!=ORI_D or img.shape[1]!=ORI_D:
                diffr = ORI_D-img.shape[0] # must >= 0
                diffc = ORI_D-img.shape[1] # must >= 0
                padr1 = np.random.randint(diffr+1)
                padr2 = diffr-padr1
                padc1 = np.random.randint(diffc+1)
                padc2 = diffc-padc1
                if np.random.rand() < 0.4:
                    img = np.pad(img, ((padr1,padr2),(padc1,padc2)), 'wrap')
                    lab = np.pad(lab, ((padr1,padr2),(padc1,padc2)), 'wrap')
                else:
                    img = np.pad(img, ((padr1,padr2),(padc1,padc2)), 'constant', constant_values=np.random.randint(7))
                    lab = np.pad(lab, ((padr1,padr2),(padc1,padc2)), 'constant', constant_values=0)

            dat_que[n,...,0] = img
            lab_que[n,...,0] = lab
        return dat_que, lab_que

In [ ]:
import bisect
N_BINS = 10
coverage = np.asarray([ x.sum() / (ORI_D**2) for x in Y_train ])
coverage_lev = np.linspace(0, 1, N_BINS)
depth_lev = np.linspace(np.min(depth_csv.z), np.max(depth_csv.z), N_BINS)
stratify = []
for cov, z in zip(coverage, depth_csv.z):
    cov_idx = bisect.bisect_left(coverage_lev, cov)
    z_idx = bisect.bisect_left(depth_lev, z)
    stratify.append(np.array([cov_idx, z_idx], dtype=np.int32))
stratify = np.asarray(stratify, dtype=np.int32)


In [ ]:
# https://www.surveysystem.com/sscalc.htm
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=2000, shuffle=True, stratify = stratify)
del coverage, coverage_lev, stratify

In [ ]:
train_generator = data_generator(X_train, Y_train, batch_size=BS, training=True)

In [ ]:
import multiprocessing
from keras.optimizers import Adam
checkpointer_best = ModelCheckpoint('./models/best.h5', monitor='val_score', mode='max', verbose=1, save_best_only=True)

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                              patience=2, min_lr=0.00001, mode='min')
# early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10)
model.compile(loss=custom_loss, optimizer=Adam(lr=0.001), metrics=[score])
history = model.fit_generator(train_generator,
                              steps_per_epoch= len(train_generator), 
                              epochs=EPOCHS_1, 
                              validation_data=(np.asarray(X_val)[...,np.newaxis], np.asarray(Y_val)[...,np.newaxis]), 
                              callbacks=[checkpointer_best, reduce_lr],
                              workers= max(1, multiprocessing.cpu_count()-3),
                              use_multiprocessing= True,
                              shuffle = True
                              )

In [ ]:
plt.plot(history.history['score'])
plt.plot(history.history['val_score'])
plt.title('LB')
plt.ylabel('LB @threshold=0.0')
plt.xlabel('epoch')
plt.legend(['train','valid'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','valid'], loc='upper right')
plt.show()

In [ ]:
model.load_weights('./models/best.h5')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.8,
                              patience=2, min_lr=0.00001, mode='min')
# early_stop = EarlyStopping(monitor='val_loss', mode='min', patience=10)
model.compile(loss=lovasz_loss, optimizer=Adam(lr=0.001), metrics=[score])
history = model.fit_generator(train_generator,
                              steps_per_epoch= len(train_generator), 
                              epochs=EPOCHS_2, 
                              validation_data=(np.asarray(X_val)[...,np.newaxis], np.asarray(Y_val)[...,np.newaxis]), 
                              callbacks=[checkpointer_best, reduce_lr],
                              workers= max(1, multiprocessing.cpu_count()-3),
                              use_multiprocessing= True,
                              shuffle = True
                              )

In [ ]:
plt.plot(history.history['score'])
plt.plot(history.history['val_score'])
plt.title('LB')
plt.ylabel('LB @threshold=0.0')
plt.xlabel('epoch')
plt.legend(['train','valid'], loc='lower right')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','valid'], loc='upper right')
plt.show()

In [ ]:
model.load_weights('./models/best.h5')
preds_val = model.predict(np.asarray(X_val)[...,np.newaxis], batch_size=BS, verbose=1)

In [ ]:
threshold_choices = np.linspace(-0.1, 0.1, 200)
best_threshold = 0.1
best_score = 0

for th in threshold_choices:
    Ys = np.squeeze(Y_val)
    Ps = np.squeeze(preds_val)
    score = score_numpy(Ys, Ps, th)
    #print('score: {:.4f} @threshold: {:.2f}'.format(score, th))
    if score>best_score:
        best_score = score
        best_threshold = th
print('best threshold: {:.2f}, best score: {:.8f}'.format(best_threshold, best_score))


In [ ]:
# Perform a sanity check on some random training samples
ix = np.random.randint(0, len(preds_val))
print(ix)
shape = Y_val[ix].shape[:2]
plt.imshow(X_val[ix])
plt.show()
plt.imshow(Y_val[ix])
plt.show()
preds = np.squeeze(preds_val[ix])
preds = preds>best_threshold
plt.imshow(preds)
plt.show()

In [ ]:
del X_train, Y_train
gc.collect()

In [ ]:
sub_rles = []
x_test = []
print('Reading testing data...')
for n, id_ in tqdm(enumerate(test_csv.id), total=len(test_csv.id)):
    path = TEST_PATH
    img = imread(path + 'images/' + id_ + '.png')[...,0].astype(np.uint8)[...,np.newaxis]
    x_test.append(img)

print('Predicting...')
preds = model.predict(np.asarray(x_test), batch_size=BS, verbose=1)[...,0]
print('Convert to RLE...')
for pred in tqdm(preds, total=len(preds)):
    pred_thresholded = (pred>best_threshold).astype(np.bool)
    rle = rle_encoding(pred_thresholded)
    sub_rles.append(rle)
sub = pd.DataFrame()
sub['id'] = test_csv.id
sub['rle_mask'] = sub_rles
sub.to_csv('salty.csv', index=False)
print('Done!')